In [ ]:
pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 2.4 MB/s eta 0:00:00


In [ ]:
pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import torch
import json
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
import pandas as pd

df = pd.read_csv('/content/train100.csv', encoding='unicode_escape')

# def process_value(value):
#     if value == 0:
#         return 0.0
#     elif value == 1:
#         return 1.0
#     else:
#         return value

# # Apply the function to each element in the DataFrame
# df = df.applymap(process_value)
df = df.astype(str)

In [ ]:

# Sample questions generation
questions = []
answers = []
coordinates = []
float_answers = []

# Define sample questions and their corresponding columns
sample_questions = [
    ("What is the phone number of the provider \"{}\"?", "Phone1_Number", "Provider Name", "single"),
    ("What is the website URL for \"{}\"?", "WebsiteUrl", "Provider Name", "single"),
    ("What is the postal code for \"{}\"?", "PostalCode", "Provider Name", "single"),
    ("Which provider in \"{}\" offers services?", "Provider City", "Provider Name", "single-city"),
    ("What type of phone is listed as the main contact for \"{}\"?", "Phone1_Type", "Provider Name", "single"),
    #("Which provider in \"{}\" accepts cash?", "Provider City", "Private Pay: Cash", "Provider Name", "single-city"),
    #("Which provider in \"{}\" accepts Medicaid?", "Provider City", "Medicaid", "Provider Name", "single-city"),
    #("Which provider in \"{}\" accepts private insurance?", "Provider City", "Private Insurance", "Provider Name", "single-city"),
    ("What is the address of \"{}\"?", "Provider Address", "Provider Name", "single"),
    ("What is the city for the provider \"{}\"?", "Provider City", "Provider Name", "single"),
    #("Which provider in \"{}\" offers independent living services?", "Provider City", "Independent Living", "Provider Name", "single-city"),
    # ("Which provider in \"{}\" offers services for traumatic brain injury?", "Provider City", "Traumatic Brain Injury / Spinal Cord Injury", "Provider Name", "single-city"),
    # ("Which provider in \"{}\" offers services for intellectual disability?", "Provider City", "Intellectual Disability / Developmental Disability", "Provider Name", "single-city"),
    # ("Which provider in \"{}\" offers bridge to independence services?", "Provider City", "Bridge to Independence", "Provider Name", "single-city"),
    # ("Which provider in \"{}\" offers private pay by credit?", "Provider City", "Private Pay: Credit", "Provider Name", "single-city"),
    # ("Which provider in \"{}\" offers private pay by check?", "Provider City", "Private Pay: Check", "Provider Name", "single-city"),
    # ("Which provider in \"{}\" offers other payment options?", "Provider City", "Other", "Provider Name", "single-city"),
    # ("What is the phone type for the provider \"{}\"?", "Phone1_Type", "Provider Name", "single"),
    # ("What is the second phone number for the provider \"{}\"?", "Phone2_Number", "Provider Name", "single"),
    # ("What is the state for the provider \"{}\"?", "Provider State", "Provider Name", "single"),
    # ("Which provider in \"{}\" accepts private pay?", "Provider City", "Private Pay: Cash", "Provider Name", "single-city"),
    # ("What is the ID for the provider \"{}\"?", "ID", "Provider Name", "single"),
    # ("Which provider in \"{}\" offers adult day care services?", "Provider City", "Adult Day Care", "Provider Name", "single-city"),
    # ("Which provider in \"{}\" offers assisted living services?", "Provider City", "Assisted Living", "Provider Name", "single-city"),
    # ("Which provider in \"{}\" offers hospice services?", "Provider City", "Hospice", "Provider Name", "single-city"),
    # ("Which provider in \"{}\" offers nursing services?", "Provider City", "Nursing", "Provider Name", "single-city"),
    # ("Which provider in \"{}\" offers physical therapy services?", "Provider City", "Physical Therapy", "Provider Name", "single-city"),
    # ("Which provider in \"{}\" offers occupational therapy services?", "Provider City", "Occupational Therapy", "Provider Name", "single-city"),
    # ("Which provider in \"{}\" offers speech therapy services?", "Provider City", "Speech Therapy", "Provider Name", "single-city"),
    # ("Which provider in \"{}\" offers skilled nursing services?", "Provider City", "Skilled Nursing", "Provider Name", "single-city")
]

# Generate questions for each provider in the table
for question_template, answer_col, provider_col, question_type in sample_questions:
    if question_type == "single":
        for index, row in df.iterrows():
            if pd.notna(row[answer_col]) and row[answer_col] != '':
                question = question_template.format(row[provider_col])
                answer = row[answer_col]
                questions.append(question)
                answers.append(answer)
                coordinates.append([(index, df.columns.get_loc(answer_col))])
                float_answers.append([float('nan')])
    elif question_type == "single-city":
        grouped = df.groupby(answer_col).apply(lambda x: x[x[provider_col] == 'Yes'][provider_col].tolist()).reset_index(name='Providers')
        for index, row in grouped.iterrows():
            if row['Providers']:  # Ensure there are providers that accept cash
                question = question_template.format(row[answer_col])
                answer_list = row['Providers']  # This will be a list of provider names
                answer_text = ', '.join(answer_list)  # Convert list to a comma-separated string for answer text
                questions.append(question)
                answers.append(answer_text)
                coordinates.append([(index, df.columns.get_loc(answer_col))])
                float_answers.append([float('nan')])

# Create a DataFrame for the generated QA pairs
qa_df = pd.DataFrame({
    'question': questions,
    'answer_text': answers,
    'answer_coordinates': coordinates,
    'float_answer': float_answers
})

# Shuffle and reset index of the DataFrame
train_df = qa_df.sample(frac=1.0, random_state=42, replace=False).reset_index(drop=True)
# Add a 'position' column to the train_df DataFrame
train_df['position'] = range(len(train_df))
batch =df[:20]
# Limit to 30 questions
train_df = train_df[:7]
train_df = train_df.reset_index(drop=True)

train_df

,question,answer_text,answer_coordinates,float_answer,position
0,"What is the phone number of the provider ""Oxfo...",6017072738,"[(1226, 7)]",[nan],0
1,"What is the postal code for ""Newton Family Med...",39345,"[(1151, 5)]",[nan],1
2,"What is the address of ""Southern Family Urgent...",1212 East Pass Road,"[(1539, 2)]",[nan],2
3,"What is the postal code for ""Lincare, Incorpor...",38732,"[(906, 5)]",[nan],3
4,"What is the phone number of the provider ""Youn...",6017854948,"[(1737, 7)]",[nan],4


In [ ]:
from transformers import TapasTokenizer
df = df[:100]
# Sample questions generation
questions = []
answers = []
coordinates = []
float_answers = []

# Define sample questions and their corresponding columns
sample_questions = [
    ("What is the phone number of the provider \"{}\"?", "Phone1_Number", 7),
    ("Which provider is located in {}?", "Provider City", 3),
    ("What is the website URL for \"{}\"?", "WebsiteUrl", 10),
    ("What is the postal code for \"{}\"?", "PostalCode", 5),
    ("Which provider in {} offers services?", "Provider City", 3),
    ("What type of phone is listed as the main contact for \"{}\"?", "Phone1_Type", 6),
]

# Generate questions for each provider in the table
for index, row in df.iterrows():
    for question_template, answer_col, answer_coord in sample_questions:
        if pd.notna(row[answer_col]) and row[answer_col] != '':
            question = question_template.format(row['Provider Name'])
            answer = row[answer_col]
            questions.append(question)
            answers.append(answer)
            coordinates.append([(index, df.columns.get_loc(answer_col))])
            float_answers.append([float('nan')])
# Create a DataFrame for the generated QA pairs
qa_df = pd.DataFrame({
    'question': questions,
    'answer_text': answers,
    'answer_coordinates': coordinates,
    'float_answer': float_answers
})
qa_df = qa_df[5:40]
# Shuffle and reset index of the DataFrame
train_df = qa_df.sample(frac=1.0, random_state=42, replace=False).reset_index(drop=True)
# Add a 'position' column to the train_df DataFrame
train_df['position'] = range(len(train_df))

batch = df.fillna('').astype(str)
train_df

In [ ]:
import ast
questions = pd.read_csv('/content/TrainingQuestions.csv', encoding='unicode_escape')

#train_df = questions.astype(str)
train_df = questions.sample(frac=1.0, random_state=42, replace=False).reset_index(drop=True)
train_df['position'] = range(len(train_df))
train_df['answer_coordinates'] = train_df['answer_coordinates'].apply(ast.literal_eval)
batch = df
train_df


,position,question,table_file,answer_coordinates,answer_text,aggregation_label,float_answer
0,0,What is the postal code for Columbia Dialysis ...,content/data.csv,"[(47, 5)]","[""39429""]",NaN,[nan]
1,1,What is a good provider in Jackson?,content/data.csv,"[(59, 15)]","[""Compere's Nursing Home""]",NaN,[nan]
2,2,What providers in Gulfport accept insurance?,content/train100.csv,"[(18, 1), (23, 1), (27, 1), (29, 1), (34, 1)]","['Coastal County Imaging Service, LLC', 'Coast...",NaN,[nan]
3,3,Which providers are located in Belzoni?,content/data.csv,"[(90, 3)]","[""Continue Care Support Services""]",NaN,[nan]
4,4,What providers in Gulfport accept Medicare?,content/train100.csv,"[(18, 1), (23, 1), (27, 1), (34, 1)]","['Coastal County Imaging Service, LLC', 'Coast...",NaN,[nan]
5,5,What services does Community Counseling Servic...,content/data.csv,"[(64, 23), (64, 24), (64, 28)]","[""OUTDOOR SPACES AND BUILDINGS"",""RESPECT AND S...",NaN,[nan]
6,6,How many providers are in Hattiesburg?,content/data.csv,"[(1, 3), (20, 3), (35, 3), (55, 3), (56, 3), (...","[""7""]",NaN,[nan]
7,7,What providers in Laurel accept check?,content/train100.csv,"[(36, 1), (45, 1), (52, 1)]","['Cohen Podiatry Clinic Laurel', 'Colonial Cha...",NaN,[nan]
8,8,How many providers in Columbia provide Communi...,content/data.csv,"[(47, 1), (58, 1)]","[""2""]",NaN,[nan]
9,9,What providers in Starkville provide Transport...,content/data.csv,"[(55, 1)]","[""Comfort Living Personal Care Home""]",NaN,[nan]


In [ ]:
from transformers import TapasTokenizer

# Initialize the tokenizer
tokenizer = TapasTokenizer.from_pretrained("Zarcend/AFFIRM-FineTune-25")
# Tokenize the input table and questions
encoding = tokenizer(
    table=batch,
    queries=train_df['question'].tolist(),
    answer_coordinates=train_df['answer_coordinates'].tolist(),
    answer_text=train_df['answer_text'].tolist(),
    float_answer=train_df['float_answer'].tolist(),
    #truncation=True,
    padding="max_length",
    return_tensors="pt"
)
encoding.keys()

tokenizer_config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/262k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (7631 > 512). Running this sequence through the model will result in indexing errors.


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

# Create a TensorDataset and DataLoader
dataset = TensorDataset(
    encoding['input_ids'],
    encoding['attention_mask'],
    encoding['token_type_ids'],
    encoding['labels'],
)
train_dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

AttributeError: 'list' object has no attribute 'size'

In [ ]:
tokenizer = TapasTokenizer.from_pretrained("Zarcend/AFFIRM-FineTune-25")
class TableDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        table = pd.read_csv('/content/train100.csv', encoding =  'unicode_escape').astype(str) # TapasTokenizer expects the table data to be text only
        table = table.sample(frac=1.0, random_state=42, replace=False).reset_index(drop=True)
        if item.position != 0:
          # use the previous table-question pair to correctly set the prev_labels token type ids
          previous_item = self.df.iloc[idx-1]
          encoding = self.tokenizer(table=table,
                                    queries=[previous_item.question, item.question],
                                    answer_coordinates=[previous_item.answer_coordinates, item.answer_coordinates],
                                    answer_text=[previous_item.answer_text, item.answer_text],
                                    float_answer=[previous_item.float_answer, item.float_answer],
                                    padding=True,
                                    #truncation=True,
                                    return_tensors="pt"
          )
          # use encodings of second table-question pair in the batch
          encoding = {key: val[-1] for key, val in encoding.items()}
        else:
          # this means it's the first table-question pair in a sequence
          encoding = self.tokenizer(table=table,
                                    queries=item.question,
                                    answer_coordinates=item.answer_coordinates,
                                    answer_text=item.answer_text,
                                    float_answer= item.float_answer,
                                    padding=True,
                                    #truncation=True,
                                    return_tensors="pt"
          )
          # remove the batch dimension which the tokenizer adds
          encoding = {key: val.squeeze(0) for key, val in encoding.items()}
        return encoding

    def __len__(self):
        return len(self.df)
# def collate_fn(batch):
#     keys = batch[0].keys()
#     max_len = max(len(item['input_ids']) for item in batch)
#     padded_batch = {}
#     for key in keys:
#         if key == 'input_ids' or key == 'attention_mask' or key == 'token_type_ids' or key == 'labels':
#             padded_batch[key] = torch.stack([torch.nn.functional.pad(item[key], (0, max_len - len(item[key]))) for item in batch])
#         else:
#             padded_batch[key] = torch.stack([item[key] for item in batch])
#     return padded_batch

train_dataset = TableDataset(df=train_df, tokenizer=tokenizer)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2)

In [ ]:
train_dataset[0]["token_type_ids"].shape

Token indices sequence length is longer than the specified maximum sequence length for this model (7628 > 512). Running this sequence through the model will result in indexing errors.


torch.Size([7628, 7])

In [ ]:
train_dataset[1]["input_ids"].shape

torch.Size([7628])

In [ ]:
batch = next(iter(train_dataloader))

In [ ]:
batch["input_ids"].shape

In [ ]:
batch["token_type_ids"].shape

torch.Size([2, 7628, 7])

In [ ]:
tokenizer.decode(batch["input_ids"][0])

In [ ]:
#first example should not have any prev_labels set
assert batch["token_type_ids"][0][:,3].sum() == 0

In [ ]:
tokenizer.decode(batch["input_ids"][1])

In [ ]:
assert batch["labels"][0].sum() == batch["token_type_ids"][1][:,3].sum()
print(batch["token_type_ids"][1][:,3].sum())

In [ ]:
from transformers import TapasForQuestionAnswering

model = TapasForQuestionAnswering.from_pretrained("Zarcend/AFFIRM-FineTune-25")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.max_seq_length = 8192
model.to(device)

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(10):  # loop over the dataset multiple times
   print("Epoch:", epoch)
   for idx, batch in enumerate(train_dataloader):
        # get the inputs;
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        labels = batch["labels"].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids,
                       labels=labels)
        loss = outputs.loss
        print("Loss:", loss.item())
        loss.backward()
        optimizer.step()

In [ ]:
!nvidia-smi -q -d Memory | grep -A4 GPU

Attached GPUs                             : 1
GPU 00000000:00:04.0
    FB Memory Usage
        Total                             : 15360 MiB
        Reserved                          : 257 MiB
        Used                              : 15025 MiB


In [ ]:
!export TORCH_USE_CUDA_DSA=1

In [ ]:
model.save_pretrained("AFFIRM-fine-tuned-tapas-25")
tokenizer.save_pretrained("AFFIRM-fine-tuned-tapas-25")

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForTableQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("AFFIRM-fine-tuned-tapas-25")
model = AutoModelForTableQuestionAnswering.from_pretrained("AFFIRM-fine-tuned-tapas-25")

In [ ]:
!zip -r /content/AFFIRM-fine-tuned-tapas-25.zip /content/AFFIRM-fine-tuned-tapas-25

updating: content/AFFIRM-fine-tuned-tapas-25/ (stored 0%)
updating: content/AFFIRM-fine-tuned-tapas-25/config.json (deflated 60%)
updating: content/AFFIRM-fine-tuned-tapas-25/model.safetensors (deflated 7%)
updating: content/AFFIRM-fine-tuned-tapas-25/tokenizer_config.json (deflated 76%)
updating: content/AFFIRM-fine-tuned-tapas-25/special_tokens_map.json (deflated 37%)
updating: content/AFFIRM-fine-tuned-tapas-25/vocab.txt (deflated 53%)


In [ ]:
from google.colab import files
files.download("/content/AFFIRM-fine-tuned-tapas-25.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from transformers import AutoTokenizer, AutoModelForTableQuestionAnswering


tokenizer = AutoTokenizer.from_pretrained("Zarcend/AFFIRM-FineTune-25")
model = AutoModelForTableQuestionAnswering.from_pretrained("Zarcend/AFFIRM-FineTune-25")
model.max_seq_length = 8192

In [ ]:
query = "What is the address of Anderson Regional Medical Center-South Campus*"
# Encode the table and query
inputs = tokenizer(table=df, queries=query, padding="max_length", return_tensors="pt")

# Pass the inputs to the model
outputs = model(**inputs)

# Interpret the logits to get the answer
logits = outputs.logits
logits_agg = outputs.logits_aggregation

# Get the answer from the model's output
predicted_answer_coordinates, predicted_aggregation_indices = tokenizer.convert_logits_to_predictions(
    inputs,
    logits.detach(),
    logits_agg.detach()
)

answers = []
for coordinates in predicted_answer_coordinates:
    cell_values = []
    for coord in coordinates:
        try:
            cell_values.append(batch.iat[coord])
        except IndexError:
            print(f"Index {coord} is out of bounds for the DataFrame with shape {batch.shape}")
            continue
    if cell_values:
        answers.append(", ".join(cell_values))

print("Answer:", answers)


Token indices sequence length is longer than the specified maximum sequence length for this model (2664 > 512). Running this sequence through the model will result in indexing errors.
